In [1]:
%matplotlib inline


Sequence Models and Long-Short Term Memory Networks
===================================================

At this point, we have seen various feed-forward networks. That is,
there is no state maintained by the network at all. This might not be
the behavior we want. Sequence models are central to NLP: they are
models where there is some sort of dependence through time between your
inputs. The classical example of a sequence model is the Hidden Markov
Model for part-of-speech tagging. Another example is the conditional
random field.

A recurrent neural network is a network that maintains some kind of
state. For example, its output could be used as part of the next input,
so that information can propogate along as the network passes over the
sequence. In the case of an LSTM, for each element in the sequence,
there is a corresponding *hidden state* $h_t$, which in principle
can contain information from arbitrary points earlier in the sequence.
We can use the hidden state to predict words in a language model,
part-of-speech tags, and a myriad of other things.


**LSTM's in Pytorch**


Before getting to the example, note a few things. Pytorch's LSTM expects
all of its inputs to be 3D tensors. The semantics of the axes of these
tensors is important. The first axis is the sequence itself, the second
indexes instances in the mini-batch, and the third indexes elements of
the input. We haven't discussed mini-batching, so lets just ignore that
and assume we will always have just 1 dimension on the second axis. If
we want to run the sequence model over the sentence "The cow jumped",
our input should look like

\begin{align}\begin{bmatrix}
   \overbrace{q_\text{The}}^\text{row vector} \\
   q_\text{cow} \\
   q_\text{jumped}
   \end{bmatrix}\end{align}

Except remember there is an additional 2nd dimension with size 1.

In addition, you could go through the sequence one at a time, in which
case the 1st axis will have size 1 also.

Let's see a quick example.



In [2]:
# Author: Robert Guthrie

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

torch.manual_seed(1)

In [31]:
lstm = nn.LSTM(3, 3) # Input dim is 3, output dim is 3
inputs = [torch.randn(1,3) for _ in range(5)] # make a sequence of length 5
inputs

[tensor([[ 0.4075, -1.2245,  1.9481]]),
 tensor([[ 1.6815,  0.6686, -1.3771]]),
 tensor([[-0.4192, -1.4726, -0.4775]]),
 tensor([[ 0.5501, -1.1967,  0.0489]]),
 tensor([[-1.8084,  2.0058,  0.9735]])]

In [32]:
hidden = (torch.randn(1,1,3),
          torch.randn(1,1,3))

for i in inputs:
    # Step through the sequence one element at a time
    # after each step, hidden contains the hidden state
    out, hidden = lstm(i.view(1, 1, -1), hidden)
    print(out, hidden)

tensor([[[ 0.1804,  0.4294, -0.1102]]], grad_fn=<StackBackward>) (tensor([[[ 0.1804,  0.4294, -0.1102]]], grad_fn=<StackBackward>), tensor([[[ 0.2202,  1.6339, -0.5626]]], grad_fn=<StackBackward>))
tensor([[[-0.0235,  0.4852, -0.2349]]], grad_fn=<StackBackward>) (tensor([[[-0.0235,  0.4852, -0.2349]]], grad_fn=<StackBackward>), tensor([[[-0.1852,  0.8546, -0.8602]]], grad_fn=<StackBackward>))
tensor([[[-0.0100,  0.4939, -0.3233]]], grad_fn=<StackBackward>) (tensor([[[-0.0100,  0.4939, -0.3233]]], grad_fn=<StackBackward>), tensor([[[-0.0246,  0.8564, -0.6756]]], grad_fn=<StackBackward>))
tensor([[[ 0.0742,  0.3390, -0.2105]]], grad_fn=<StackBackward>) (tensor([[[ 0.0742,  0.3390, -0.2105]]], grad_fn=<StackBackward>), tensor([[[ 0.2014,  0.6834, -0.6234]]], grad_fn=<StackBackward>))
tensor([[[0.0873, 0.0805, 0.1141]]], grad_fn=<StackBackward>) (tensor([[[0.0873, 0.0805, 0.1141]]], grad_fn=<StackBackward>), tensor([[[0.1656, 0.1017, 0.1917]]], grad_fn=<StackBackward>))


Alternatively, we can do the entire sequence all at once.
the first value returned by LSTM is all of the hidden states throughout
the sequence. the second is just the most recent hidden state
(compare the last slice of "out" with "hidden" below, they are the same)
The reason for this is that:
"out" will give you access to all hidden states in the sequence
"hidden" will allow you to continue the sequence and backpropagate,
by passing it as an argument  to the lstm at a later time
Add the extra 2nd dimension

In [33]:
inputs = torch.cat(inputs).view(len(inputs), 1, -1)
hidden = (torch.randn(1, 1, 3), torch.randn(1, 1, 3)) # clean
out, hidden = lstm(inputs, hidden)
print(out,'\n\n',hidden)

tensor([[[ 0.3352,  0.1937,  0.2125]],

        [[ 0.0083,  0.3226, -0.0826]],

        [[ 0.0669,  0.3911, -0.1468]],

        [[ 0.1299,  0.2975, -0.1315]],

        [[ 0.1270,  0.0655,  0.1405]]], grad_fn=<StackBackward>) 

 (tensor([[[0.1270, 0.0655, 0.1405]]], grad_fn=<StackBackward>), tensor([[[0.2381, 0.0824, 0.2392]]], grad_fn=<StackBackward>))


**Example: An LSTM for Part-of-Speech Tagging**


In this section, we will use an LSTM to get part of speech tags. We will
not use Viterbi or Forward-Backward or anything like that, but as a
(challenging) exercise to the reader, think about how Viterbi could be
used after you have seen what is going on.

The model is as follows: let our input sentence be
$w_1, \dots, w_M$, where $w_i \in V$, our vocab. Also, let
$T$ be our tag set, and $y_i$ the tag of word $w_i$.
Denote our prediction of the tag of word $w_i$ by
$\hat{y}_i$.

This is a structure prediction, model, where our output is a sequence
$\hat{y}_1, \dots, \hat{y}_M$, where $\hat{y}_i \in T$.

To do the prediction, pass an LSTM over the sentence. Denote the hidden
state at timestep $i$ as $h_i$. Also, assign each tag a
unique index (like how we had word\_to\_ix in the word embeddings
section). Then our prediction rule for $\hat{y}_i$ is

\begin{align}\hat{y}_i = \text{argmax}_j \  (\log \text{Softmax}(Ah_i + b))_j\end{align}

That is, take the log softmax of the affine map of the hidden state,
and the predicted tag is the tag that has the maximum value in this
vector. Note this implies immediately that the dimensionality of the
target space of $A$ is $|T|$.


Prepare data:



In [36]:
def prepare_sequence(seq, to_ix):
    idxs = [to_ix[w] for w in seq]
    return torch.tensor(idxs, dtype=torch.long)

training_data = [
    ("The dog ate the apple".split(), ["DET", "NN", "V", "DET", "NN"]),
    ("Everybody read that book".split(), ["NN", "V", "DET", "NN"])
]
word_to_ix = {}
for sent, tags in training_data:
    for word in sent:
        if word not in word_to_ix:
            word_to_ix[word] = len(word_to_ix)
print(word_to_ix)

tag_to_ix = {"DET":0, "NN":1, "V":2}

EMBEDDING_DIM = 6
HIDDEN_DIM = 6

{'The': 0, 'dog': 1, 'ate': 2, 'the': 3, 'apple': 4, 'Everybody': 5, 'read': 6, 'that': 7, 'book': 8}


Create the model:



In [35]:
class LSTMTagger(nn.Module):
    def __init__(self, embedding_dim, hidden_dim, vocab_size, tagset_size):
        super(LSTMTagger, self).__init__()
        self.hidden_dim = hidden_dim
        self.word_embeddings = nn.Embedding(vocab_size, embedding_dim)
        
        self.lstm = nn.LSTM(embedding_dim, hidden_dim)
        
        self.hidden2tag = nn.Linear(hidden_dim, tagset_size)
        self.hidden = self.init_hidden()
        
        
    def init_hidden(self):
        return (torch.zeros(1, 1, self.hidden_dim), torch.zeros(1, 1, self.hidden_dim))
    
    def forward(self, sentence):
        embeds = self.word_embeddings(sentence)
        lstm_out, self.hidden = self.lstm(embeds.view(len(sentence), 1, -1), self.hidden)
        tag_space = self.hidden2tag(lstm_out.view(len(sentence), -1))
        tag_scores = F.log_softmax(tag_space, dim=1)
        return tag_scores

Train the model:



In [40]:
model = LSTMTagger(EMBEDDING_DIM, HIDDEN_DIM, len(word_to_ix), len(tag_to_ix))
loss_function = nn.NLLLoss()
optimizer = optim.SGD(model.parameters(), lr=0.1)

with torch.no_grad():
    inputs = prepare_sequence(training_data[0][0], word_to_ix)
    tag_scores = model(inputs)
    print(tag_scores)
    
for epoch in range(300):
    for sentence, tags in training_data:
        model.zero_grad()
        
        model.hidden = model.init_hidden()
        
        sentence_in = prepare_sequence(sentence, word_to_ix)
        targets = prepare_sequence(tags, tag_to_ix)
        
        tag_scores = model(sentence_in)
        
        loss = loss_function(tag_scores, targets)
        loss.backward()
        optimizer.step()
        
with torch.no_grad():
    inputs = prepare_sequence(training_data[0][0], word_to_ix)
    tag_scores = model(inputs)
    print(tag_scores)

tensor([[-1.2390, -0.9063, -1.1831],
        [-1.2718, -0.9262, -1.1282],
        [-1.1816, -1.0312, -1.0888],
        [-1.1182, -0.9727, -1.2205],
        [-1.1862, -0.9324, -1.2007]])
tensor([[-0.1768, -2.3393, -2.7230],
        [-4.2830, -0.0463, -3.4589],
        [-4.1339, -3.2676, -0.0556],
        [-0.0352, -3.8807, -4.2754],
        [-4.1215, -0.0273, -4.5369]])


**Exercise: Augmenting the LSTM part-of-speech tagger with character-level features**


In the example above, each word had an embedding, which served as the
inputs to our sequence model. Let's augment the word embeddings with a
representation derived from the characters of the word. We expect that
this should help significantly, since character-level information like
affixes have a large bearing on part-of-speech. For example, words with
the affix *-ly* are almost always tagged as adverbs in English.

To do this, let $c_w$ be the character-level representation of
word $w$. Let $x_w$ be the word embedding as before. Then
the input to our sequence model is the concatenation of $x_w$ and
$c_w$. So if $x_w$ has dimension 5, and $c_w$
dimension 3, then our LSTM should accept an input of dimension 8.

To get the character level representation, do an LSTM over the
characters of a word, and let $c_w$ be the final hidden state of
this LSTM. Hints:

* There are going to be two LSTM's in your new model.
  The original one that outputs POS tag scores, and the new one that
  outputs a character-level representation of each word.
* To do a sequence model over characters, you will have to embed characters.
  The character embeddings will be the input to the character LSTM.


